In [1]:
import pandas as pd
from pprint import pprint

In [2]:
from awudima import Federation, DataSource, DataSourceType, AwudimaFQP

In [3]:
fed = Federation("platoon_pilot1a_fed", "PLATOON Pilot1a Federation", "Federation of platoon datasets")
fed.to_json()

{'fedId': 'platoon_pilot1a_fed',
 'name': 'PLATOON Pilot1a Federation',
 'desc': 'Federation of platoon datasets',
 'rdfmts': [],
 'sources': {}}

In [4]:
ds1 = DataSource("pilot1a_engie",  # name of the database
                 'engie_pilot1a',  # Id of the data source
                 '0.0.0.0:27017',  # URL of the data source
                 DataSourceType.MONGODB_LD_FLAT,  # type of the data source
                 desc="Pilot1A data on Wind turbines",
                 params={
                     "<http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#jdbcDSN>": "pilot1a_engie",
                     'username': 'root',
                     'password': '1234'
                 },
                 acronym='ENGIE_Pilot1a_DS')
ds1.to_json()

{'dsId': 'engie_pilot1a',
 'name': 'pilot1a_engie',
 'desc': 'Pilot1A data on Wind turbines',
 'url': '0.0.0.0:27017',
 'dstype': 'MONGODB_LD_FLAT',
 'params': {'<http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#jdbcDSN>': 'pilot1a_engie',
  'username': 'root',
  'password': '1234'},
 'acronym': 'ENGIE_Pilot1a_DS',
 'labeling_property': 'http://www.w3.org/2000/01/rdf-schema#label',
 'mappings': {},
 'mapping_paths': [],
 'mappings_type': None,
 'typing_predicate': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
 'policy': None}

In [5]:
fed.addSource(ds1)

In [6]:
fed.to_json()

{'fedId': 'platoon_pilot1a_fed',
 'name': 'PLATOON Pilot1a Federation',
 'desc': 'Federation of platoon datasets',
 'rdfmts': [],
 'sources': {'engie_pilot1a': {'dsId': 'engie_pilot1a',
   'name': 'pilot1a_engie',
   'desc': 'Pilot1A data on Wind turbines',
   'url': '0.0.0.0:27017',
   'dstype': 'MONGODB_LD_FLAT',
   'params': {'<http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#jdbcDSN>': 'pilot1a_engie',
    'username': 'root',
    'password': '1234'},
   'acronym': 'ENGIE_Pilot1a_DS',
   'labeling_property': 'http://www.w3.org/2000/01/rdf-schema#label',
   'mappings': {},
   'mapping_paths': [],
   'mappings_type': None,
   'typing_predicate': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   'policy': None}}}

In [7]:
# collect RDF Molecule Templates
fed.extract_molecules()

0.0.0.0:27017
----------------------
total mts: 13
1 ) extracting description for  0.0.0.0:27017 is done!


{http://www.w3.org/2006/time#Instant,
 https://w3id.org/platoon/OnshoreWindTurbine,
 https://w3id.org/platoon/PositionEvaluation,
 https://w3id.org/platoon/PositionProperty,
 https://w3id.org/platoon/StatusCodeEvaluation,
 https://w3id.org/platoon/StatusCodeProperty,
 https://w3id.org/platoon/Vane,
 https://w3id.org/platoon/WindFarm,
 https://w3id.org/seas/AverageEvaluation,
 https://w3id.org/seas/MaximumEvaluation,
 https://w3id.org/seas/MinimumEvaluation,
 https://w3id.org/seas/StandardDeviationEvaluation}

In [8]:
fed.to_json()

{'fedId': 'platoon_pilot1a_fed',
 'name': 'PLATOON Pilot1a Federation',
 'desc': 'Federation of platoon datasets',
 'rdfmts': [{'mtId': 'https://w3id.org/platoon/PositionProperty',
   'mttype': 'typed',
   'label': 'https://w3id.org/platoon/PositionProperty',
   'desc': '',
   'cardinality': -1,
   'subClassOf': [],
   'predicates': [{'predId': 'https://w3id.org/seas/evaluation',
     'label': 'https://w3id.org/seas/evaluation',
     'desc': '',
     'cardinality': -1,
     'prefix': '',
     'ranges': ['https://w3id.org/seas/MinimumEvaluation',
      'https://w3id.org/seas/StandardDeviationEvaluation',
      'https://w3id.org/platoon/PositionEvaluation',
      'https://w3id.org/seas/MaximumEvaluation',
      'https://w3id.org/seas/AverageEvaluation'],
     'constraints': [],
     'policy': None},
    {'predId': 'http://www.w3.org/2000/01/rdf-schema#label',
     'label': 'http://www.w3.org/2000/01/rdf-schema#label',
     'desc': '',
     'cardinality': -1,
     'prefix': '',
     'rang

In [9]:
# FQP
fqp = AwudimaFQP(fed)
fqp.federation

platoon_pilot1a_fed

In [10]:
sparql_query = "SELECT distinct * WHERE {?windfarm a <https://w3id.org/platoon/WindFarm>. " \
                   "?windfarm <http://www.w3.org/2000/01/rdf-schema#label> ?label. " \
                   "}"
resultset = fqp.execute(sparql_query, keep_in_memory=True)

In [11]:
resultset.decomposition

SELECT DISTINCT *
 WHERE {  { SERVICE <pilot1a_engie.windturbine_eventlogs:ds-engie_pilot1a@0.0.0.0:27017> { 
        ?windfarm <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/platoon/WindFarm> . 
        ?windfarm <http://www.w3.org/2000/01/rdf-schema#label> ?label }   
 }

}

In [12]:
resultset.plan

 {'label', 'windfarm'}
   {'label', 'windfarm'}
     SERVICE <pilot1a_engie.windturbine_eventlogs:ds-engie_pilot1a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.label', 'null']},
                                {'$ne': ['$@graph.label', '']}]},
             '@graph.@type': {'$in': ['plt:WindFarm']}}},
 {'$project': {'_id': 0, 'label': '$@graph.label', 'windfarm': '$@graph.@id'}}]
     }

In [14]:
resultset.retrieved_results

1

In [13]:
results = resultset.results
results

{'head': {'vars': ['?windfarm', '?label']},
 'results': {'bindings': [{'windfarm': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve'},
    'label': {'type': 'literal', 'value': 'FRCVE'}}]},
 'message': 'All results are retrieved from resultset!',
 'query': 'SELECT distinct * WHERE {?windfarm a <https://w3id.org/platoon/WindFarm>. ?windfarm <http://www.w3.org/2000/01/rdf-schema#label> ?label. }'}

In [15]:
{k:v['value']  for r in resultset.get() for k, v in r.items()}

{'windfarm': 'http://engie.com/platoon/resource/windfarm/frcve',
 'label': 'FRCVE'}

In [16]:
df = pd.DataFrame(results['results']['bindings'])
df.head()

,windfarm,label
0,"{'type': 'uri', 'value': 'http://engie.com/pla...","{'type': 'literal', 'value': 'FRCVE'}"


In [ ]:
pilot1a_query1 = """

                prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT Distinct *
                WHERE{
                    ?windfarm a  <https://w3id.org/platoon/WindFarm>.
                    ?windfarm <http://www.w3.org/2000/01/rdf-schema#label> ?label.
                    
                }
            """


In [ ]:
resultset = fqp.execute(pilot1a_query1, keep_in_memory=True)
resultset.plan

In [ ]:
pprint(resultset.results)

In [17]:
pilot1a_query2 = """

                prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT Distinct *
                WHERE{
                    ?vane a  <https://w3id.org/platoon/Vane>.
                    ?vane <http://www.w3.org/2000/01/rdf-schema#label> ?label.
                    ?vane <https://w3id.org/platoon/hasStandardDeviationPosition> ?stdvPosition .
                }
            """


In [18]:
resultset = fqp.execute(pilot1a_query2, keep_in_memory=True)
resultset.plan

 {'label', 'vane', 'stdvPosition'}
   {'label', 'vane', 'stdvPosition'}
     SERVICE <pilot1a_engie.windturbine_vane:ds-engie_pilot1a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.label', 'null']},
                                {'$ne': ['$@graph.label', '']},
                                {'$ne': ['$@graph.hasStandardDeviationPosition',
                                         'null']},
                                {'$ne': ['$@graph.hasStandardDeviationPosition',
                                         '']}]},
             '@graph.@type': {'$in': ['plt:Vane']}}},
 {'$project': {'_id': 0,
               'label': '$@graph.label',
               'stdvPosition': '$@graph.hasStandardDeviationPosition',
               'vane': '$@graph.@id'}}]
     }

In [19]:
resultset.results

{'head': {'vars': ['?stdvPosition', '?vane', '?label']},
 'results': {'bindings': [{'vane': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane'},
    'label': {'type': 'literal', 'value': 'vane'},
    'stdvPosition': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane/property/position/standarddeviation'}}]},
 'message': 'All results are retrieved from resultset!',
 'query': '\n\n                prefix cim: <http://www.iec.ch/TC57/CIM#>\n                prefix plt: <https://w3id.org/platoon/>\n                prefix seas: <https://w3id.org/seas/> \n                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>\n                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>\n                prefix time: <http://www.w3.org/2006/time#>\n                SELECT Distinct *\n                WHERE{\n                    ?vane a  <https://w3id.org/platoon/Vane>.\n            

In [20]:
pilot1a_query3 = """

                prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT Distinct *
                WHERE{
                    ?vane a  <https://w3id.org/platoon/Vane>.
                    ?vane <http://www.w3.org/2000/01/rdf-schema#label> ?label.
                    ?vane <https://w3id.org/platoon/hasStandardDeviationPosition> ?stdvPosition .
                    ?vane <https://w3id.org/seas/subSystemOf> ?subSystemOf .
                    ?vane <https://w3id.org/platoon/hasAveragePosition> ?avgPosition .
                    ?vane <https://w3id.org/platoon/hasMaximumPosition> ?maxPosition .
                    ?vane <https://w3id.org/platoon/hasMinimumPosition> ?minPosition .
                    ?maxPosition <http://www.w3.org/2000/01/rdf-schema#label> ?poslabel.
                    ?maxPosition seas:evaluation ?positionEval.
                }
            """

In [21]:
resultset = fqp.execute(pilot1a_query3, keep_in_memory=True)
resultset.plan

 {'label', 'positionEval', 'maxPosition', 'avgPosition', 'vane', 'subSystemOf', 'poslabel', 'stdvPosition', 'minPosition'}
   {'label', 'positionEval', 'maxPosition', 'avgPosition', 'vane', 'subSystemOf', 'poslabel', 'stdvPosition', 'minPosition'}
     {'label', 'positionEval', 'maxPosition', 'avgPosition', 'vane', 'subSystemOf', 'poslabel', 'stdvPosition', 'minPosition'}
       SERVICE <pilot1a_engie.windturbine_vane:ds-engie_pilot1a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.label', 'null']},
                                {'$ne': ['$@graph.label', '']},
                                {'$ne': ['$@graph.hasStandardDeviationPosition',
                                         'null']},
                                {'$ne': ['$@graph.hasStandardDeviationPosition',
                                         '']},
                                {'$ne': ['$@graph.subSystemOf', 'null']},
                     

In [22]:
resultset.results

{'head': {'vars': ['?subSystemOf',
   '?poslabel',
   '?vane',
   '?positionEval',
   '?label',
   '?maxPosition',
   '?avgPosition',
   '?minPosition',
   '?stdvPosition']},
 'results': {'bindings': [{'maxPosition': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane/property/position/maximum'},
    'poslabel': {'type': 'literal', 'value': 'vane position_1_2  maximum'},
    'positionEval': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane/property/position/maximum/evaluation/2019-01-05t00:00:00z'},
    'vane': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane'},
    'label': {'type': 'literal', 'value': 'vane'},
    'stdvPosition': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/vane/property/position/standarddeviation'},
    'subSystemOf': {'type': 'uri',
     'value': 'http://eng

In [23]:
resultset.retrieved_results

1

In [32]:
query = """
  prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT Distinct *
                WHERE {     
                    
                    ?context time:inXSDDateTime ?time .
                }
"""

In [33]:
resultset = fqp.execute(query, keep_in_memory=True)
resultset.plan

 {'context', 'time'}
   {'context', 'time'}
     SERVICE <pilot1a_engie.windturbine_vane:ds-engie_pilot1a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.inXSDDateTime', 'null']},
                                {'$ne': ['$@graph.inXSDDateTime', '']}]},
             '@graph.@type': {'$in': ['time:Instant']}}},
 {'$unionWith': {'coll': 'windturbine_eventlogs',
                 'pipeline': [{'$unwind': '$@graph'},
                              {'$unwind': '$@graph.@type'},
                              {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.inXSDDateTime',
                                                                      'null']},
                                                             {'$ne': ['$@graph.inXSDDateTime',
                                                                      '']}]},
                                          '@graph.@type': {'$in': ['time:Instant']}}},
             

In [34]:
resultset.decomposition


prefix cim: <http://www.iec.ch/TC57/CIM#>
prefix plt: <https://w3id.org/platoon/>
prefix seas: <https://w3id.org/seas/>
prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
prefix time: <http://www.w3.org/2006/time#>
SELECT DISTINCT *
 WHERE {  { SERVICE <pilot1a_engie.windturbine_vane:ds-engie_pilot1a@0.0.0.0:27017> { 
        ?context time:inXSDDateTime ?time }   
 }

}

In [36]:
resultset.results

{'head': {'vars': ['?context', '?time']},
 'results': {'bindings': [{'context': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/windfarm/frcve/windturbine/80499/timestamp/2019-01-05t00:00:00z'},
    'time': {'type': 'literal',
     'value': '2019-01-05T00:00:00Z',
     'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime'}}]},
 'message': 'All results are retrieved from resultset!',
 'query': '\n  prefix cim: <http://www.iec.ch/TC57/CIM#>\n                prefix plt: <https://w3id.org/platoon/>\n                prefix seas: <https://w3id.org/seas/> \n                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>\n                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>\n                prefix time: <http://www.w3.org/2006/time#>\n                SELECT Distinct *\n                WHERE {\n                    \n                    ?context time:inXSDDateTime ?time .\n                }\n'}

In [38]:
fed2 = Federation("platoon_pilot3a_fed", "PLATOON Pilot3a Federation", "Federation of platoon datasets")
ds1 = DataSource("pilot3a_engie",  # name of the database
                 'engie_pilot3a',  # Id of the data source
                 '0.0.0.0:27017',  # URL of the data source
                 DataSourceType.MONGODB_LD_FLAT,  # type of the data source
                 desc="Pilot3A data on Wind turbines",
                 params={
                     "<http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#jdbcDSN>": "pilot3a_engie",
                     'username': 'root',
                     'password': '1234'
                 },
                 acronym='ENGIE_Pilot3a_DS')
ds1.to_json()
fed2.addSource(ds1)
fed2.to_json()

{'fedId': 'platoon_pilot3a_fed',
 'name': 'PLATOON Pilot3a Federation',
 'desc': 'Federation of platoon datasets',
 'rdfmts': [],
 'sources': {'engie_pilot3a': {'dsId': 'engie_pilot3a',
   'name': 'pilot3a_engie',
   'desc': 'Pilot3A data on Wind turbines',
   'url': '0.0.0.0:27017',
   'dstype': 'MONGODB_LD_FLAT',
   'params': {'<http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#jdbcDSN>': 'pilot3a_engie',
    'username': 'root',
    'password': '1234'},
   'acronym': 'ENGIE_Pilot3a_DS',
   'labeling_property': 'http://www.w3.org/2000/01/rdf-schema#label',
   'mappings': {},
   'mapping_paths': [],
   'mappings_type': None,
   'typing_predicate': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   'policy': None}}}

In [39]:
fed2.extract_molecules()

0.0.0.0:27017
----------------------
total mts: 22
1 ) extracting description for  0.0.0.0:27017 is done!


{http://www.w3.org/2006/time#Instant,
 http://www.w3.org/ns/prov#Activity,
 https://w3id.org/bot#Building,
 https://w3id.org/bot#Zone,
 https://w3id.org/pep/ProcedureExecution,
 https://w3id.org/platoon/AirTemperatureEvaluation,
 https://w3id.org/platoon/CloudOpacityEvaluation,
 https://w3id.org/platoon/CloudOpacityProperty,
 https://w3id.org/platoon/DiffuseSolarRadiationEvaluation,
 https://w3id.org/platoon/DiffuseSolarRadiationProperty,
 https://w3id.org/platoon/DirectSolarRadiationEvaluation,
 https://w3id.org/platoon/DirectSolarRadiationProperty,
 https://w3id.org/platoon/ElectricityEnergyConsumptionEvaluation,
 https://w3id.org/platoon/HeatingGasEnergyConsumptionEvaluation,
 https://w3id.org/platoon/OccupiedNumberEvaluation,
 https://w3id.org/platoon/OpeningPercentageEvaluation,
 https://w3id.org/platoon/SolarRadiationEvaluation,
 https://w3id.org/platoon/SolarRadiationProperty,
 https://w3id.org/saref#Occupancy}

In [40]:
# FQP
fqp = AwudimaFQP(fed2)
fqp.federation

platoon_pilot3a_fed

In [41]:

pilot3a_query1 = """
    prefix cim: <http://www.iec.ch/TC57/CIM#>
    prefix plt: <https://w3id.org/platoon/>
    prefix seas: <https://w3id.org/seas/> 
    prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
    prefix time: <http://www.w3.org/2006/time#>
    SELECT Distinct *
    WHERE{
        ?building <https://w3id.org/platoon/hasCloudOpacity> ?cop .   
        ?building <https://w3id.org/bot#containsZone> ?zone .
        ?building <https://w3id.org/platoon/hasDirectNormalIrradiance> ?irradiance         
    }
"""
rs = fqp.execute(pilot3a_query1, keep_in_memory=True)
rs.results

{'head': {'vars': ['?irradiance', '?zone', '?cop', '?building']},
 'results': {'bindings': [{'building': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/building/crigen-stains'},
    'cop': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/building/crigen-stains/property/cloudopacity'},
    'irradiance': {'type': 'uri',
     'value': 'http://engie.com/platoon/resource/building/crigen-stains/property/directnormalirradiance'},
    'zone': {'type': 'uri', 'value': ''}}]},
 'message': 'All results are retrieved from resultset!',
 'query': '\n    prefix cim: <http://www.iec.ch/TC57/CIM#>\n    prefix plt: <https://w3id.org/platoon/>\n    prefix seas: <https://w3id.org/seas/> \n    prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>\n    prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>\n    prefix time: <http://www.w3.org/2006/time#>\n    SELECT Distinct *\n    WHERE{\n        ?building <https://w3id.org/platoon/hasCloudOpacity> ?cop .   \n        ?

In [42]:
rs.plan

 {'building', 'cop', 'zone', 'irradiance'}
   {'building', 'cop', 'zone', 'irradiance'}
     SERVICE <pilot3a_engie.weather-estimated-actuals:ds-engie_pilot3a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.hasCloudOpacity', 'null']},
                                {'$ne': ['$@graph.hasCloudOpacity', '']},
                                {'$ne': ['$@graph.hasDirectNormalIrradiance',
                                         'null']},
                                {'$ne': ['$@graph.hasDirectNormalIrradiance',
                                         '']}]},
             '@graph.@type': {'$in': ['bot:Building']}}},
 {'$unionWith': {'coll': 'occupancy',
                 'pipeline': [{'$unwind': '$@graph'},
                              {'$unwind': '$@graph.@type'},
                              {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.containsZone',
                                                       

In [44]:
pilot3a_query2 = """
            prefix cim: <http://www.iec.ch/TC57/CIM#>
            prefix plt: <https://w3id.org/platoon/>
            prefix seas: <https://w3id.org/seas/> 
            prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
            prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
            prefix time: <http://www.w3.org/2006/time#>
            SELECT Distinct *
            WHERE{                
                ?eval seas:evaluationOf ?cop .
                ?eval <https://w3id.org/seas/evaluatedSimpleValue> ?value.      
                ?eval <https://w3id.org/seas/hasTemporalContext> ?tcontext.      
                ?tcontext <http://www.w3.org/2006/time#inXSDDateTime> ?inxsdtime.    
            }
        """
rs = fqp.execute(pilot3a_query2, keep_in_memory=True)
rs.plan

 {'tcontext', 'value', 'cop', 'eval', 'inxsdtime'}
   {'tcontext', 'value', 'cop', 'eval', 'inxsdtime'}
     {'tcontext', 'value', 'cop', 'eval', 'inxsdtime'}
       SERVICE <pilot3a_engie.bms-controller:ds-engie_pilot3a@0.0.0.0:27017> { 
[{'$unwind': '$@graph'},
 {'$unwind': '$@graph.@type'},
 {'$match': {'$expr': {'$and': [{'$ne': ['$@graph.evaluationOf', 'null']},
                                {'$ne': ['$@graph.evaluationOf', '']},
                                {'$ne': ['$@graph.evaluatedSimpleValue',
                                         'null']},
                                {'$ne': ['$@graph.evaluatedSimpleValue', '']},
                                {'$ne': ['$@graph.hasTemporalContext', 'null']},
                                {'$ne': ['$@graph.hasTemporalContext', '']}]},
             '@graph.@type': {'$in': ['plt:AirTemperatureEvaluation']}}},
 {'$project': {'_id': 0,
               'cop': '$@graph.evaluationOf',
               'eval': '$@graph.@id',
             

In [ ]:
 pilot3a_query2 = """
            prefix cim: <http://www.iec.ch/TC57/CIM#>
            prefix plt: <https://w3id.org/platoon/>
            prefix seas: <https://w3id.org/seas/> 
            prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
            prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
            prefix time: <http://www.w3.org/2006/time#>
            SELECT Distinct *
            WHERE{                
                ?eval seas:evaluationOf ?cop .
                ?eval <https://w3id.org/seas/evaluatedSimpleValue> ?value.      
                ?eval <https://w3id.org/seas/hasTemporalContext> ?tcontext.      
                ?tcontext <http://www.w3.org/2006/time#inXSDDateTime> ?inxsdtime.    
            }
        """
    pilot3a_query3 = """
            prefix cim: <http://www.iec.ch/TC57/CIM#>
            prefix plt: <https://w3id.org/platoon/>
            prefix seas: <https://w3id.org/seas/> 
            prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
            prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
            prefix time: <http://www.w3.org/2006/time#>
            SELECT Distinct *
            WHERE{
                ?building <https://w3id.org/platoon/hasGlobalHorizontalIrradiance> ?prop .
                ?prop a <https://w3id.org/platoon/SolarRadiationProperty> .
                ?prop <https://w3id.org/seas/evaluation> ?eval.
                ?eval a plt:SolarRadiationEvaluation.
                ?eval <https://w3id.org/seas/evaluatedSimpleValue> ?value.
                ?eval <http://www.w3.org/2006/time#duration> ?duration.
            }
        """
    pilot3a_query4 = """
    
        prefix cim: <http://www.iec.ch/TC57/CIM#>
            prefix plt: <https://w3id.org/platoon/>
            prefix seas: <https://w3id.org/seas/> 
            prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
            prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
            prefix time: <http://www.w3.org/2006/time#>
            SELECT Distinct *
            WHERE{
                ?building <https://w3id.org/platoon/hasCloudOpacity> ?cop .
                ?building <https://w3id.org/platoon/hasDirectNormalIrradiance> ?directNormalIrradiance.
                ?building <https://w3id.org/bot#containsZone> ?zone.
                ?building <http://www.w3.org/2000/01/rdf-schema#label> ?label.
                ?building <https://w3id.org/platoon/hasGlobalHorizontalIrradiance> ?globalHorIrradiance.
                ?building <https://w3id.org/platoon/hasDirectHorizontalIrradiance> ?directHorIrradiance.
                ?building <https://w3id.org/platoon/hasDiffuseHorizontalIrradiance> ?diffHorIrradiance.
            }
    """
    pilot3a_query5 = """

            prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT Distinct *
                WHERE{
                    ?building <https://w3id.org/bot#containsZone> ?zone.
                    ?building <http://www.w3.org/2000/01/rdf-schema#label> ?label.
                }
        """
    pilot3a_query6 = """
                prefix cim: <http://www.iec.ch/TC57/CIM#>
                prefix plt: <https://w3id.org/platoon/>
                prefix seas: <https://w3id.org/seas/> 
                prefix wgs84_pos: <http://www.w3.org/2003/01/geo/wgs84_pos#>
                prefix qudt: <http://www.qudt.org/2.1/schema/qudt/>
                prefix time: <http://www.w3.org/2006/time#>
                SELECT *
                WHERE{                
                        
                    ?tcontext <http://www.w3.org/2006/time#inXSDDateTime> ?inxsdtime.    
                }
            """